# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
fct_guest_spending['visit_date']=pd.to_datetime(fct_guest_spending['visit_date'])
july_fct_guest_spending = fct_guest_spending[(fct_guest_spending['visit_date']>=
                                              '2024-07-01')&(fct_guest_spending['visit_date']<
                                              '2024-08-01')].copy()
all_experience_type=fct_guest_spending['park_experience_type'].unique()
july_avg_spend=july_fct_guest_spending.groupby('park_experience_type').agg(Avg_spend_park=('amount_spent','mean'))
final_report=july_avg_spend.reindex(all_experience_type,fill_value=0.0)
print(final_report)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Convert date column
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

# Filter August 2024 visits
august_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= '2024-08-01') &
    (fct_guest_spending['visit_date'] < '2024-09-01')
].copy()

# Sort by guest_id and visit_date (so we know first and last)
august_data = august_data.sort_values(['guest_id', 'visit_date'])

# Aggregate spending per guest per visit_date (in case multiple transactions per day)
guest_daily_spend = august_data.groupby(['guest_id','visit_date']).agg(
    total_spend=('amount_spent','sum')
).reset_index()

# For each guest, find first and last visit
first_last_spend = guest_daily_spend.groupby('guest_id').agg(
    first_visit=('total_spend','first'),
    last_visit=('total_spend','last'),
    visit_count=('visit_date','count')
).reset_index()

# Keep only guests with more than 1 visit
multi_visitors = first_last_spend[first_last_spend['visit_count'] > 1].copy()

# Calculate difference
multi_visitors['spending_diff'] = multi_visitors['last_visit'] - multi_visitors['first_visit']

print(multi_visitors)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Convert date column
fct_guest_spending['visit_date']=pd.to_datetime(fct_guest_spending['visit_date'])

# Filter September 2024 visits
september_data = fct_guest_spending[(fct_guest_spending['visit_date']>='2024-09-01')
  & (fct_guest_spending['visit_date']<'2024-10-01')].copy()

total_spend_guest = september_data.groupby('guest_id').agg(
  total_spend=('amount_spent','sum')).reset_index()

total_spend_guest = total_spend_guest[total_spend_guest['total_spend']>0]

def categoryspend(total_spend):
  if (total_spend>=0) and (total_spend<50):
      return 'Low'
  elif (total_spend>=50) and (total_spend<100):
      return 'Medium'
  else:
      return 'High'

total_spend_guest['spend_segment'] = total_spend_guest['total_spend'].apply(categoryspend)
print(total_spend_guest)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)